xAI Grok 4 is released on 2025-07-10, let us try it out

- https://grok.com/

In [1]:
from openai import OpenAI
import os 

In [2]:
client = OpenAI(
  api_key=os.getenv("XAI_API_KEY"),
  base_url="https://api.x.ai/v1",
)

In [3]:
def ask_grok(prompt, client=None, model="grok-4", base_url="https://api.x.ai/v1"):
    if client is None:
        client = OpenAI(
            api_key=os.getenv("XAI_API_KEY"),
            base_url=base_url,
        )
        
    response = client.chat.completions.create(
        model=model,  # "grok-3",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return parse_grok_response(response=response)

In [4]:
# Grok 4 Response Parser and Usage Examples

def parse_grok_response(response):
    """
    Parse Grok 4 ChatCompletion response to extract key information
    
    Args:
        response: ChatCompletion object from Grok 4 API
        
    Returns:
        dict: Parsed response data
    """
    try:
        # Extract message content
        message_content = response.choices[0].message.content
        
        # Extract model name
        model = response.model
        
        # Extract total tokens
        total_tokens = response.usage.total_tokens
        
        # Additional useful information
        parsed_data = {
            'message_content': message_content,
            'model': model,
            'total_tokens': total_tokens,
            'prompt_tokens': response.usage.prompt_tokens,
            'completion_tokens': response.usage.completion_tokens,
            'reasoning_tokens': response.usage.completion_tokens_details.get('reasoning_tokens', 0),
            'finish_reason': response.choices[0].finish_reason,
            'response_id': response.id,
            'created': response.created
        }
        
        return parsed_data
        
    except (AttributeError, IndexError, KeyError) as e:
        print(f"Error parsing response: {e}")
        return None

In [7]:
prompt = "5 的阶乘是多少？"

In [8]:
completion = client.chat.completions.create(
  model="grok-4",  # "grok-3",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

In [ ]:
resp = parse_grok_response(response=completion)

In [10]:
resp.update({"prompt": prompt})

In [11]:
resp 

{'message_content': '5 的阶乘（5!）是 5 × 4 × 3 × 2 × 1 = **120**。  \n\n如果你有其他数字的阶乘或其他问题，随时问我！😊',
 'model': 'grok-4-0709',
 'total_tokens': 163,
 'prompt_tokens': 15,
 'completion_tokens': 45,
 'reasoning_tokens': 103,
 'finish_reason': 'stop',
 'response_id': 'db1a4cf8-b289-31a1-ed34-e65f2c4c4097',
 'created': 1752148189,
 'prompt': '5 的阶乘是多少？'}

In [14]:
prompt = "用Python写一个函数来计算斐波那契数列的第n项"
resp = ask_grok(prompt, client=client)

In [16]:
resp

{'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
 'model': 'grok-4-0709',
 'total_tokens': 697,
 'prompt_tokens': 24,
 'completion_tokens': 301,
 'reasoning_tokens': 372,
 'finish_reason': 'stop',
 'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
 'created': 1752

In [5]:
import re
from IPython.display import display, Markdown, HTML
try:
    from IPython.display import Code
except ImportError:
    # Fallback for older IPython versions
    Code = None

def display_grok_response(parsed_response, show_stats=True):
    """
    Display Grok response content nicely in Jupyter Notebook
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        show_stats (bool): Whether to show token usage statistics
    """
    
    content = parsed_response.get('message_content', '')
    
    if show_stats:
        # Display token usage statistics
        stats_html = f"""
        <div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
            <h4>📊 Token Usage Statistics</h4>
            <ul>
                <li><strong>Model:</strong> {parsed_response.get('model', 'N/A')}</li>
                <li><strong>Total Tokens:</strong> {parsed_response.get('total_tokens', 0)}</li>
                <li><strong>Prompt Tokens:</strong> {parsed_response.get('prompt_tokens', 0)}</li>
                <li><strong>Completion Tokens:</strong> {parsed_response.get('completion_tokens', 0)}</li>
                <li><strong>Reasoning Tokens:</strong> {parsed_response.get('reasoning_tokens', 0)}</li>
                <li><strong>Finish Reason:</strong> {parsed_response.get('finish_reason', 'N/A')}</li>
            </ul>
        </div>
        """
        display(HTML(stats_html))
    
    # Display the main content
    display(Markdown("### 🤖 Grok Response:"))
    display(Markdown(content))

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def display_code_blocks(parsed_response):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            display(Code(block['code'], language=block['language']))
        else:
            # Fallback: display as preformatted text with basic styling
            code_html = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
            display(HTML(code_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

# Enhanced display function with everything
def display_grok_analysis(parsed_response, extract_code=True, show_stats=True):
    """
    Complete analysis and display of Grok response
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        show_stats (bool): Whether to show token usage statistics
    """
    
    # Show statistics
    if show_stats:
        display_grok_response(parsed_response, show_stats=True)
    else:
        display_grok_response(parsed_response, show_stats=False)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response)

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with full analysis
    display_grok_analysis(fibonacci_response)
    
    # Extract code for testing
    code = quick_test_code(fibonacci_response)
    print("\n" + "="*50)
    print("🚀 Ready to test! Use this code:")
    print("="*50)
    return code

# Simple usage examples
"""
# Basic usage:
display_grok_response(your_parsed_response)

# With code extraction:
display_grok_analysis(your_parsed_response)

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)  # Execute the code

# Save code to file:
extract_and_save_code(your_parsed_response, "fibonacci.py")
"""

'\n# Basic usage:\ndisplay_grok_response(your_parsed_response)\n\n# With code extraction:\ndisplay_grok_analysis(your_parsed_response)\n\n# Just extract code for testing:\ncode = quick_test_code(your_parsed_response)\nexec(code)  # Execute the code\n\n# Save code to file:\nextract_and_save_code(your_parsed_response, "fibonacci.py")\n'

In [19]:
display_grok_response(resp)

### 🤖 Grok Response:

```python
def fibonacci(n):
    """
    计算斐波那契数列的第n项（从0开始索引）。
    例如：
    - fibonacci(0) = 0
    - fibonacci(1) = 1
    - fibonacci(2) = 1
    - fibonacci(3) = 2
    - fibonacci(4) = 3
    - fibonacci(5) = 5

    参数:
    n (int): 非负整数，表示斐波那契数列的索引。

    返回:
    int: 斐波那契数列的第n项。

    异常:
    ValueError: 如果n为负数。
    """
    if n < 0:
        raise ValueError("n must be a non-negative integer")
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

# 示例用法
if __name__ == "__main__":
    print(fibonacci(0))  # 输出: 0
    print(fibonacci(1))  # 输出: 1
    print(fibonacci(5))  # 输出: 5
    print(fibonacci(10)) # 输出: 55
```

In [6]:
def fibonacci(n):
    """
    计算斐波那契数列的第n项（从0开始索引）。
    例如：
    - fibonacci(0) = 0
    - fibonacci(1) = 1
    - fibonacci(2) = 1
    - fibonacci(3) = 2
    - fibonacci(4) = 3
    - fibonacci(5) = 5

    参数:
    n (int): 非负整数，表示斐波那契数列的索引。

    返回:
    int: 斐波那契数列的第n项。

    异常:
    ValueError: 如果n为负数。
    """
    if n < 0:
        raise ValueError("n must be a non-negative integer")
    if n == 0:
        return 0
    if n == 1:
        return 1

    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

# 示例用法
if __name__ == "__main__":
    print(fibonacci(0))  # 输出: 0
    print(fibonacci(1))  # 输出: 1
    print(fibonacci(5))  # 输出: 5
    print(fibonacci(10)) # 输出: 55

0
1
5
55


In [7]:
# prompt = "When was the largest firework show in Japan on July 4th"

prompt = "What is square root of 2"

resp = ask_grok(prompt, client=client)
display_grok_response(resp)

### 🤖 Grok Response:

Based on your query, I'll assume you're asking for the mathematical value of the square root of 2 (often written as √2). Let me break it down step by step, including the exact value, an approximation, and some context!

### 1. **Exact Value**
- The square root of 2 is the number that, when multiplied by itself, equals 2. In other words, if \( x = \sqrt{2} \), then \( x \times x = 2 \).
- It's an **irrational number**, meaning it can't be expressed as a simple fraction (like 1/2 or 3/4) and its decimal goes on forever without repeating.
- Exact mathematical notation: \( \sqrt{2} \) or \( 2^{1/2} \).

### 2. **Approximate Value**
- A common decimal approximation is **1.41421356237** (rounded to 11 decimal places).
- For everyday use:
  - To 2 decimals: ≈ 1.41
  - To 3 decimals: ≈ 1.414
  - To 5 decimals: ≈ 1.41421
- You can get more precise values using a calculator or tools like Python/Google/Wolfram Alpha.

### 3. **How to Calculate It**
- **Manually**: You can use methods like the Babylonian algorithm (a type of iterative approximation). Start with a guess (e.g., 1.4), square it, and refine:
  - Guess: 1.4 → 1.4² = 1.96 (too low)
  - Refine: Average of 1.4 and (2/1.4) ≈ 1.414 → 1.414² ≈ 1.999 (very close!)
  - Repeat for more precision.
- **Using a Calculator**: Just type √2 or "sqrt(2)" into most calculators or search engines.
- **In Programming**: In Python, it's `math.sqrt(2)` which gives ~1.4142135623730951.

### 4. **Fun Facts and Context**
- √2 is famous in math because it's the length of the diagonal of a square with sides of length 1 (from the Pythagorean theorem: diagonal = √(1² + 1²) = √2).
- It was proven irrational by ancient Greek mathematicians (like Hippasus) around 500 BC, which was a big deal at the time!
- It appears in nature, geometry, and physics (e.g., in wave patterns or the golden ratio approximations).

If this isn't what you meant (e.g., if you wanted the square root of a different number, or something like "square root of 2" in a non-math context like a riddle or code), feel free to provide more details! 😊

In [8]:
prompt = "创建一个比较不同数值积分方法的表格，包括梯形法则、辛普森法则等，并给出Python实现。"

resp = ask_grok(prompt, client=client)
display_grok_response(resp)

### 🤖 Grok Response:

以下是针对用户查询的完整响应。我将首先创建一个表格，比较几种常见的数值积分方法，包括梯形法则（Trapezoidal Rule）和辛普森法则（Simpson's Rule）。为了使比较更全面，我还包括了矩形法则（Rectangular Rule，使用左黎曼和）和中点法则（Midpoint Rule）。这些方法都是基于牛顿-科特斯公式，用于近似计算定积分 ∫_a^b f(x) dx。

表格比较的方面包括：方法描述、误差阶（大O表示）、优点和缺点。这些比较假设函数f(x)是连续且光滑的，n是区间的分区数。

### 数值积分方法比较表格

| 方法名称          | 描述                                                                 | 误差阶          | 优点                                                                 | 缺点                                                                 |
|-------------------|----------------------------------------------------------------------|-----------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| **矩形法则 (Rectangular Rule, Left Riemann Sum)** | 使用左端点高度的矩形近似每个子区间的面积。公式：h * Σ f(a + (i-1)h) for i=1 to n。 | O(h) 或 O(1/n) | 简单易实现；计算量小。                                               | 准确性低，尤其是对于非线性函数；误差较大。                           |
| **中点法则 (Midpoint Rule)** | 使用每个子区间中点的函数值作为高度的矩形近似。公式：h * Σ f(a + (i-0.5)h) for i=1 to n。 | O(h²) 或 O(1/n²) | 比矩形法则更准确；对线性函数精确。                                   | 仍需函数在子区间中点求值；对于奇异函数可能不稳定。                   |
| **梯形法则 (Trapezoidal Rule)** | 使用梯形近似每个子区间（连接端点）。公式：(h/2) * (f(a) + 2Σ f(a+ih) + f(b)) for i=1 to n-1。 | O(h²) 或 O(1/n²) | 简单且准确度较高；对线性函数精确；易于并行化。                       | 对于高频振荡函数误差可能放大；需要端点值。                           |
| **辛普森法则 (Simpson's Rule)** | 使用抛物线近似每两个子区间。公式：(h/3) * (f(a) + 4Σ f(a+(2i-1)h) + 2Σ f(a+2ih) + f(b))，n须为偶数。 | O(h⁴) 或 O(1/n⁴) | 高准确性（四阶误差）；对多项式函数特别有效；收敛快。                 | 需要偶数分区（n even）；计算更复杂；对非光滑函数敏感。               |

**说明**：
- **h** 是子区间宽度：h = (b - a) / n。
- **误差阶** 表示局部误差随h减小而减小的速度。高阶方法（如辛普森）在相同n下更准确，但计算成本更高。
- **适用条件**：所有方法都适用于连续函数；辛普森需要函数至少三次可微以达到理论误差。
- **一般建议**：对于粗略估计，使用梯形；对于高精度，使用辛普森或自适应版本。

### Python实现
以下是为每个方法提供的Python实现。这些函数假设输入是一个可调用的函数`f`（例如，lambda x: x**2）、积分区间`[a, b]`和分区数`n`。我使用纯Python（无外部库）以保持简单。如果n较大，建议使用NumPy加速。

```python
import math  # 用于示例函数，如果需要

# 示例函数：积分 ∫_0^1 x^2 dx = 1/3 ≈ 0.3333
def f(x):
    return x ** 2

# 1. 矩形法则 (Left Riemann Sum)
def rectangular_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        integral += f(a + i * h)  # 使用左端点
    return integral * h

# 2. 中点法则
def midpoint_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        midpoint = a + (i + 0.5) * h
        integral += f(midpoint)
    return integral * h

# 3. 梯形法则
def trapezoidal_rule(f, a, b, n):
    h = (b - a) / n
    integral = (f(a) + f(b)) / 2.0
    for i in range(1, n):
        integral += f(a + i * h)
    return integral * h

# 4. 辛普森法则 (n 必须为偶数)
def simpsons_rule(f, a, b, n):
    if n % 2 != 0:
        raise ValueError("n must be even for Simpson's Rule")
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n, 2):  # 奇数点：系数4
        integral += 4 * f(a + i * h)
    for i in range(2, n-1, 2):  # 偶数点：系数2
        integral += 2 * f(a + i * h)
    return integral * h / 3.0

# 测试示例
a, b, n = 0, 1, 100  # n=100，对于辛普森需偶数
print("Rectangular:", rectangular_rule(f, a, b, n))  # ≈0.32835
print("Midpoint:", midpoint_rule(f, a, b, n))        # ≈0.333325
print("Trapezoidal:", trapezoidal_rule(f, a, b, n))  # ≈0.33335
print("Simpson's:", simpsons_rule(f, a, b, n))       # ≈0.333333
```

**测试结果解释**：
- 真实值：1/3 ≈ 0.333333。
- 以n=100为例，辛普森法则最接近真实值，体现了其高阶准确性。
- 你可以替换`f`为其他函数，并调整n来观察收敛性。

如果需要更多方法（如Romberg积分或自适应辛普森）、错误处理、或使用NumPy的优化版本，请随时告知！

In [9]:
import re
from IPython.display import display, Markdown, HTML
try:
    from IPython.display import Code
except ImportError:
    # Fallback for older IPython versions
    Code = None

def display_grok_response(parsed_response, show_stats=True, collapsible_stats=False):
    """
    Display Grok response content nicely in Jupyter Notebook
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        show_stats (bool): Whether to show token usage statistics
        collapsible_stats (bool): Whether to make stats collapsible
    """
    
    content = parsed_response.get('message_content', '')
    
    if show_stats:
        # Create collapsible stats section
        stats_content = f"""
        <ul style="margin: 10px 0;">
            <li><strong>Model:</strong> {parsed_response.get('model', 'N/A')}</li>
            <li><strong>Total Tokens:</strong> {parsed_response.get('total_tokens', 0)}</li>
            <li><strong>Prompt Tokens:</strong> {parsed_response.get('prompt_tokens', 0)}</li>
            <li><strong>Completion Tokens:</strong> {parsed_response.get('completion_tokens', 0)}</li>
            <li><strong>Reasoning Tokens:</strong> {parsed_response.get('reasoning_tokens', 0)}</li>
            <li><strong>Finish Reason:</strong> {parsed_response.get('finish_reason', 'N/A')}</li>
        </ul>
        """
        
        if collapsible_stats:
            stats_html = f"""
            <details style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
                <summary style="cursor: pointer; font-weight: bold; font-size: 16px;">📊 Token Usage Statistics (Click to expand)</summary>
                {stats_content}
            </details>
            """
        else:
            stats_html = f"""
            <div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
                <h4>📊 Token Usage Statistics</h4>
                {stats_content}
            </div>
            """
        display(HTML(stats_html))
    
    # Display the main content with LaTeX support
    display(Markdown("### 🤖 Grok Response:"))
    display_content_with_latex(content)

def display_content_with_latex(content):
    """
    Display content with LaTeX math rendering support
    
    Args:
        content (str): Content that may contain LaTeX math
    """
    
    # Check if content contains LaTeX patterns
    latex_patterns = [
        r'\$\$.*?\$\$',  # Display math
        r'\$.*?\$',      # Inline math
        r'\\begin\{.*?\}.*?\\end\{.*?\}',  # LaTeX environments
        r'\\[a-zA-Z]+\{.*?\}',  # LaTeX commands
    ]
    
    has_latex = any(re.search(pattern, content, re.DOTALL) for pattern in latex_patterns)
    
    if has_latex:
        # Enable MathJax for LaTeX rendering
        mathjax_config = """
        <script>
        window.MathJax = {
            tex: {
                inlineMath: [['$', '$'], ['\\(', '\\)']],
                displayMath: [['$$', '$$'], ['\\[', '\\]']],
                processEscapes: true,
                processEnvironments: true
            },
            options: {
                ignoreHtmlClass: 'tex2jax_ignore',
                processHtmlClass: 'tex2jax_process'
            }
        };
        </script>
        <script type="text/javascript" id="MathJax-script" async
            src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
        </script>
        """
        display(HTML(mathjax_config))
        
        # Display with MathJax processing
        display(HTML(f'<div class="tex2jax_process">{content}</div>'))
    else:
        # Regular markdown display
        display(Markdown(content))

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def extract_tables(content):
    """
    Extract and enhance markdown tables
    
    Args:
        content (str): Content with potential tables
        
    Returns:
        list: List of table HTML strings
    """
    # Pattern to match markdown tables
    table_pattern = r'(\|.*?\|(?:\r?\n|\r)\|.*?\|(?:(?:\r?\n|\r)\|.*?\|)*)'
    matches = re.findall(table_pattern, content, re.MULTILINE)
    
    enhanced_tables = []
    for match in matches:
        # Convert markdown table to HTML with better styling
        lines = match.strip().split('\n')
        if len(lines) < 2:
            continue
            
        # Parse header
        header = [cell.strip() for cell in lines[0].split('|')[1:-1]]
        
        # Skip separator line
        if len(lines) < 3:
            continue
            
        # Parse data rows
        rows = []
        for line in lines[2:]:
            if line.strip():
                row = [cell.strip() for cell in line.split('|')[1:-1]]
                rows.append(row)
        
        # Generate styled HTML table
        table_html = """
        <table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
            <thead style="background-color: #f2f2f2;">
                <tr>
        """
        
        for cell in header:
            table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{cell}</th>'
        
        table_html += "</tr></thead><tbody>"
        
        for i, row in enumerate(rows):
            bg_color = "#f9f9f9" if i % 2 == 0 else "#ffffff"
            table_html += f'<tr style="background-color: {bg_color};">'
            for cell in row:
                table_html += f'<td style="border: 1px solid #ddd; padding: 8px;">{cell}</td>'
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        enhanced_tables.append(table_html)
    
    return enhanced_tables

def display_code_blocks(parsed_response, collapsible=False):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        collapsible (bool): Whether to make code blocks collapsible
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        code_display = ""
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            code_display = f'<div id="code-block-{i}"></div>'
        else:
            # Fallback: display as preformatted text with basic styling
            code_display = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
        
        if collapsible:
            display(HTML(f"""
            <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; padding: 10px; background-color: #f0f0f0; 
                               font-weight: bold;">📝 Code Block {i} ({block['language']}) - Click to expand</summary>
                <div style="padding: 10px;">
                    {code_display}
                </div>
            </details>
            """))
        else:
            display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
            display(HTML(code_display))
        
        # Display actual code if using Code class
        if Code:
            display(Code(block['code'], language=block['language']))

def display_enhanced_tables(parsed_response):
    """
    Extract and display enhanced tables
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    tables = extract_tables(content)
    
    if not tables:
        print("No tables found in the response.")
        return
    
    for i, table_html in enumerate(tables, 1):
        display(HTML(f"<h4>📊 Table {i}</h4>"))
        display(HTML(table_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

def display_grok_analysis(parsed_response, extract_code=True, extract_tables=True, 
                         show_stats=True, collapsible=True):
    """
    Complete analysis and display of Grok response with all enhancements
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        extract_tables (bool): Whether to extract and display tables separately
        show_stats (bool): Whether to show token usage statistics
        collapsible (bool): Whether to make sections collapsible
    """
    
    # Show main response with LaTeX support
    display_grok_response(parsed_response, show_stats=show_stats, collapsible_stats=collapsible)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response, collapsible=collapsible)
    
    # Extract and display tables
    if extract_tables:
        print("\n" + "="*50)
        display(HTML("<h3>📊 Table Enhancement</h3>"))
        display_enhanced_tables(parsed_response)

def create_comprehensive_display(parsed_response, title="Grok Response Analysis"):
    """
    Create a comprehensive, collapsible display of all response components
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        title (str): Title for the display
    """
    
    # Main container with collapsible sections
    display(HTML(f"""
    <div style="border: 2px solid #007acc; border-radius: 10px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);">
        <h2 style="text-align: center; color: #007acc; margin: 0; padding: 20px;">
            🤖 {title}
        </h2>
    </div>
    """))
    
    # Stats section (collapsible)
    display_grok_response(parsed_response, show_stats=True, collapsible_stats=True)
    
    # Main content with LaTeX support
    content = parsed_response.get('message_content', '')
    display(HTML("""
    <details open style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
        <summary style="cursor: pointer; padding: 10px; background-color: #e8f4f8; 
                       font-weight: bold; font-size: 16px;">📝 Full Response Content</summary>
        <div style="padding: 10px;">
    """))
    
    display_content_with_latex(content)
    display(HTML("</div></details>"))
    
    # Code blocks (collapsible)
    code_blocks = extract_code_blocks(content)
    if code_blocks:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f0f8e8; 
                           font-weight: bold; font-size: 16px;">🔧 Code Blocks</summary>
            <div style="padding: 10px;">
        """))
        display_code_blocks(parsed_response, collapsible=False)
        display(HTML("</div></details>"))
    
    # Tables (collapsible)
    tables = extract_tables(content)
    if tables:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f8f0e8; 
                           font-weight: bold; font-size: 16px;">📊 Enhanced Tables</summary>
            <div style="padding: 10px;">
        """))
        display_enhanced_tables(parsed_response)
        display(HTML("</div></details>"))

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the enhanced helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with comprehensive analysis
    create_comprehensive_display(fibonacci_response, "Fibonacci Function Analysis")

# Enhanced usage examples
"""
# Basic usage with LaTeX support:
display_grok_response(your_parsed_response)

# Full analysis with collapsible sections:
display_grok_analysis(your_parsed_response, collapsible=True)

# Comprehensive display with all features:
create_comprehensive_display(your_parsed_response, "My Analysis")

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)

# Save code to file:
extract_and_save_code(your_parsed_response, "my_code.py")
"""

'\n# Basic usage with LaTeX support:\ndisplay_grok_response(your_parsed_response)\n\n# Full analysis with collapsible sections:\ndisplay_grok_analysis(your_parsed_response, collapsible=True)\n\n# Comprehensive display with all features:\ncreate_comprehensive_display(your_parsed_response, "My Analysis")\n\n# Just extract code for testing:\ncode = quick_test_code(your_parsed_response)\nexec(code)\n\n# Save code to file:\nextract_and_save_code(your_parsed_response, "my_code.py")\n'

In [10]:
create_comprehensive_display(resp, "Physics Problem Analysis")

### 🤖 Grok Response:

以下是针对用户查询的完整响应。我将首先创建一个表格，比较几种常见的数值积分方法，包括梯形法则（Trapezoidal Rule）和辛普森法则（Simpson's Rule）。为了使比较更全面，我还包括了矩形法则（Rectangular Rule，使用左黎曼和）和中点法则（Midpoint Rule）。这些方法都是基于牛顿-科特斯公式，用于近似计算定积分 ∫_a^b f(x) dx。

表格比较的方面包括：方法描述、误差阶（大O表示）、优点和缺点。这些比较假设函数f(x)是连续且光滑的，n是区间的分区数。

### 数值积分方法比较表格

| 方法名称          | 描述                                                                 | 误差阶          | 优点                                                                 | 缺点                                                                 |
|-------------------|----------------------------------------------------------------------|-----------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| **矩形法则 (Rectangular Rule, Left Riemann Sum)** | 使用左端点高度的矩形近似每个子区间的面积。公式：h * Σ f(a + (i-1)h) for i=1 to n。 | O(h) 或 O(1/n) | 简单易实现；计算量小。                                               | 准确性低，尤其是对于非线性函数；误差较大。                           |
| **中点法则 (Midpoint Rule)** | 使用每个子区间中点的函数值作为高度的矩形近似。公式：h * Σ f(a + (i-0.5)h) for i=1 to n。 | O(h²) 或 O(1/n²) | 比矩形法则更准确；对线性函数精确。                                   | 仍需函数在子区间中点求值；对于奇异函数可能不稳定。                   |
| **梯形法则 (Trapezoidal Rule)** | 使用梯形近似每个子区间（连接端点）。公式：(h/2) * (f(a) + 2Σ f(a+ih) + f(b)) for i=1 to n-1。 | O(h²) 或 O(1/n²) | 简单且准确度较高；对线性函数精确；易于并行化。                       | 对于高频振荡函数误差可能放大；需要端点值。                           |
| **辛普森法则 (Simpson's Rule)** | 使用抛物线近似每两个子区间。公式：(h/3) * (f(a) + 4Σ f(a+(2i-1)h) + 2Σ f(a+2ih) + f(b))，n须为偶数。 | O(h⁴) 或 O(1/n⁴) | 高准确性（四阶误差）；对多项式函数特别有效；收敛快。                 | 需要偶数分区（n even）；计算更复杂；对非光滑函数敏感。               |

**说明**：
- **h** 是子区间宽度：h = (b - a) / n。
- **误差阶** 表示局部误差随h减小而减小的速度。高阶方法（如辛普森）在相同n下更准确，但计算成本更高。
- **适用条件**：所有方法都适用于连续函数；辛普森需要函数至少三次可微以达到理论误差。
- **一般建议**：对于粗略估计，使用梯形；对于高精度，使用辛普森或自适应版本。

### Python实现
以下是为每个方法提供的Python实现。这些函数假设输入是一个可调用的函数`f`（例如，lambda x: x**2）、积分区间`[a, b]`和分区数`n`。我使用纯Python（无外部库）以保持简单。如果n较大，建议使用NumPy加速。

```python
import math  # 用于示例函数，如果需要

# 示例函数：积分 ∫_0^1 x^2 dx = 1/3 ≈ 0.3333
def f(x):
    return x ** 2

# 1. 矩形法则 (Left Riemann Sum)
def rectangular_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        integral += f(a + i * h)  # 使用左端点
    return integral * h

# 2. 中点法则
def midpoint_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        midpoint = a + (i + 0.5) * h
        integral += f(midpoint)
    return integral * h

# 3. 梯形法则
def trapezoidal_rule(f, a, b, n):
    h = (b - a) / n
    integral = (f(a) + f(b)) / 2.0
    for i in range(1, n):
        integral += f(a + i * h)
    return integral * h

# 4. 辛普森法则 (n 必须为偶数)
def simpsons_rule(f, a, b, n):
    if n % 2 != 0:
        raise ValueError("n must be even for Simpson's Rule")
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n, 2):  # 奇数点：系数4
        integral += 4 * f(a + i * h)
    for i in range(2, n-1, 2):  # 偶数点：系数2
        integral += 2 * f(a + i * h)
    return integral * h / 3.0

# 测试示例
a, b, n = 0, 1, 100  # n=100，对于辛普森需偶数
print("Rectangular:", rectangular_rule(f, a, b, n))  # ≈0.32835
print("Midpoint:", midpoint_rule(f, a, b, n))        # ≈0.333325
print("Trapezoidal:", trapezoidal_rule(f, a, b, n))  # ≈0.33335
print("Simpson's:", simpsons_rule(f, a, b, n))       # ≈0.333333
```

**测试结果解释**：
- 真实值：1/3 ≈ 0.333333。
- 以n=100为例，辛普森法则最接近真实值，体现了其高阶准确性。
- 你可以替换`f`为其他函数，并调整n来观察收敛性。

如果需要更多方法（如Romberg积分或自适应辛普森）、错误处理、或使用NumPy的优化版本，请随时告知！

以下是针对用户查询的完整响应。我将首先创建一个表格，比较几种常见的数值积分方法，包括梯形法则（Trapezoidal Rule）和辛普森法则（Simpson's Rule）。为了使比较更全面，我还包括了矩形法则（Rectangular Rule，使用左黎曼和）和中点法则（Midpoint Rule）。这些方法都是基于牛顿-科特斯公式，用于近似计算定积分 ∫_a^b f(x) dx。

表格比较的方面包括：方法描述、误差阶（大O表示）、优点和缺点。这些比较假设函数f(x)是连续且光滑的，n是区间的分区数。

### 数值积分方法比较表格

| 方法名称          | 描述                                                                 | 误差阶          | 优点                                                                 | 缺点                                                                 |
|-------------------|----------------------------------------------------------------------|-----------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| **矩形法则 (Rectangular Rule, Left Riemann Sum)** | 使用左端点高度的矩形近似每个子区间的面积。公式：h * Σ f(a + (i-1)h) for i=1 to n。 | O(h) 或 O(1/n) | 简单易实现；计算量小。                                               | 准确性低，尤其是对于非线性函数；误差较大。                           |
| **中点法则 (Midpoint Rule)** | 使用每个子区间中点的函数值作为高度的矩形近似。公式：h * Σ f(a + (i-0.5)h) for i=1 to n。 | O(h²) 或 O(1/n²) | 比矩形法则更准确；对线性函数精确。                                   | 仍需函数在子区间中点求值；对于奇异函数可能不稳定。                   |
| **梯形法则 (Trapezoidal Rule)** | 使用梯形近似每个子区间（连接端点）。公式：(h/2) * (f(a) + 2Σ f(a+ih) + f(b)) for i=1 to n-1。 | O(h²) 或 O(1/n²) | 简单且准确度较高；对线性函数精确；易于并行化。                       | 对于高频振荡函数误差可能放大；需要端点值。                           |
| **辛普森法则 (Simpson's Rule)** | 使用抛物线近似每两个子区间。公式：(h/3) * (f(a) + 4Σ f(a+(2i-1)h) + 2Σ f(a+2ih) + f(b))，n须为偶数。 | O(h⁴) 或 O(1/n⁴) | 高准确性（四阶误差）；对多项式函数特别有效；收敛快。                 | 需要偶数分区（n even）；计算更复杂；对非光滑函数敏感。               |

**说明**：
- **h** 是子区间宽度：h = (b - a) / n。
- **误差阶** 表示局部误差随h减小而减小的速度。高阶方法（如辛普森）在相同n下更准确，但计算成本更高。
- **适用条件**：所有方法都适用于连续函数；辛普森需要函数至少三次可微以达到理论误差。
- **一般建议**：对于粗略估计，使用梯形；对于高精度，使用辛普森或自适应版本。

### Python实现
以下是为每个方法提供的Python实现。这些函数假设输入是一个可调用的函数`f`（例如，lambda x: x**2）、积分区间`[a, b]`和分区数`n`。我使用纯Python（无外部库）以保持简单。如果n较大，建议使用NumPy加速。

```python
import math  # 用于示例函数，如果需要

# 示例函数：积分 ∫_0^1 x^2 dx = 1/3 ≈ 0.3333
def f(x):
    return x ** 2

# 1. 矩形法则 (Left Riemann Sum)
def rectangular_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        integral += f(a + i * h)  # 使用左端点
    return integral * h

# 2. 中点法则
def midpoint_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        midpoint = a + (i + 0.5) * h
        integral += f(midpoint)
    return integral * h

# 3. 梯形法则
def trapezoidal_rule(f, a, b, n):
    h = (b - a) / n
    integral = (f(a) + f(b)) / 2.0
    for i in range(1, n):
        integral += f(a + i * h)
    return integral * h

# 4. 辛普森法则 (n 必须为偶数)
def simpsons_rule(f, a, b, n):
    if n % 2 != 0:
        raise ValueError("n must be even for Simpson's Rule")
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n, 2):  # 奇数点：系数4
        integral += 4 * f(a + i * h)
    for i in range(2, n-1, 2):  # 偶数点：系数2
        integral += 2 * f(a + i * h)
    return integral * h / 3.0

# 测试示例
a, b, n = 0, 1, 100  # n=100，对于辛普森需偶数
print("Rectangular:", rectangular_rule(f, a, b, n))  # ≈0.32835
print("Midpoint:", midpoint_rule(f, a, b, n))        # ≈0.333325
print("Trapezoidal:", trapezoidal_rule(f, a, b, n))  # ≈0.33335
print("Simpson's:", simpsons_rule(f, a, b, n))       # ≈0.333333
```

**测试结果解释**：
- 真实值：1/3 ≈ 0.333333。
- 以n=100为例，辛普森法则最接近真实值，体现了其高阶准确性。
- 你可以替换`f`为其他函数，并调整n来观察收敛性。

如果需要更多方法（如Romberg积分或自适应辛普森）、错误处理、或使用NumPy的优化版本，请随时告知！

import math  # 用于示例函数，如果需要

# 示例函数：积分 ∫_0^1 x^2 dx = 1/3 ≈ 0.3333
def f(x):
    return x ** 2

# 1. 矩形法则 (Left Riemann Sum)
def rectangular_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        integral += f(a + i * h)  # 使用左端点
    return integral * h

# 2. 中点法则
def midpoint_rule(f, a, b, n):
    h = (b - a) / n
    integral = 0.0
    for i in range(n):
        midpoint = a + (i + 0.5) * h
        integral += f(midpoint)
    return integral * h

# 3. 梯形法则
def trapezoidal_rule(f, a, b, n):
    h = (b - a) / n
    integral = (f(a) + f(b)) / 2.0
    for i in range(1, n):
        integral += f(a + i * h)
    return integral * h

# 4. 辛普森法则 (n 必须为偶数)
def simpsons_rule(f, a, b, n):
    if n % 2 != 0:
        raise ValueError("n must be even for Simpson's Rule")
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n, 2):  # 奇数点：系数4
        integral += 4 * f(a + i * h)
    for i in range(2, n-1, 2):  # 偶数点：系数2
        integral += 2 * f(a + i * h)
    return integral * h / 3.0

# 测试示例
a, b, n = 0, 1, 100  # n=100，对于辛普森需偶数
print("Rectangular:", rectangular_rule(f, a, b, n))  # ≈0.32835
print("Midpoint:", midpoint_rule(f, a, b, n))        # ≈0.333325
print("Trapezoidal:", trapezoidal_rule(f, a, b, n))  # ≈0.33335
print("Simpson's:", simpsons_rule(f, a, b, n))       # ≈0.333333

In [16]:
import re
from IPython.display import display, Markdown, HTML
try:
    from IPython.display import Code
except ImportError:
    # Fallback for older IPython versions
    Code = None

def display_grok_response(parsed_response, show_stats=True, collapsible_stats=False):
    """
    Display Grok response content nicely in Jupyter Notebook
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        show_stats (bool): Whether to show token usage statistics
        collapsible_stats (bool): Whether to make stats collapsible
    """
    
    content = parsed_response.get('message_content', '')
    
    if show_stats:
        # Create collapsible stats section
        stats_content = f"""
        <ul style="margin: 10px 0;">
            <li><strong>Model:</strong> {parsed_response.get('model', 'N/A')}</li>
            <li><strong>Total Tokens:</strong> {parsed_response.get('total_tokens', 0)}</li>
            <li><strong>Prompt Tokens:</strong> {parsed_response.get('prompt_tokens', 0)}</li>
            <li><strong>Completion Tokens:</strong> {parsed_response.get('completion_tokens', 0)}</li>
            <li><strong>Reasoning Tokens:</strong> {parsed_response.get('reasoning_tokens', 0)}</li>
            <li><strong>Finish Reason:</strong> {parsed_response.get('finish_reason', 'N/A')}</li>
        </ul>
        """
        
        if collapsible_stats:
            stats_html = f"""
            <details style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
                <summary style="cursor: pointer; font-weight: bold; font-size: 16px;">📊 Token Usage Statistics (Click to expand)</summary>
                {stats_content}
            </details>
            """
        else:
            stats_html = f"""
            <div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
                <h4>📊 Token Usage Statistics</h4>
                {stats_content}
            </div>
            """
        display(HTML(stats_html))
    
    # Display the main content with LaTeX support
    display(Markdown("### 🤖 Grok Response:"))
    display_content_with_latex(content)

def display_content_with_latex(content):
    """
    Display content with LaTeX math rendering support
    
    Args:
        content (str): Content that may contain LaTeX math
    """
    
    # Check if content contains LaTeX patterns (including old-style notation)
    latex_patterns = [
        r'\$\$.*?\$\

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def extract_tables(content):
    """
    Extract and enhance markdown tables
    
    Args:
        content (str): Content with potential tables
        
    Returns:
        list: List of table HTML strings
    """
    # Pattern to match markdown tables
    table_pattern = r'(\|.*?\|(?:\r?\n|\r)\|.*?\|(?:(?:\r?\n|\r)\|.*?\|)*)'
    matches = re.findall(table_pattern, content, re.MULTILINE)
    
    enhanced_tables = []
    for match in matches:
        # Convert markdown table to HTML with better styling
        lines = match.strip().split('\n')
        if len(lines) < 2:
            continue
            
        # Parse header
        header = [cell.strip() for cell in lines[0].split('|')[1:-1]]
        
        # Skip separator line
        if len(lines) < 3:
            continue
            
        # Parse data rows
        rows = []
        for line in lines[2:]:
            if line.strip():
                row = [cell.strip() for cell in line.split('|')[1:-1]]
                rows.append(row)
        
        # Generate styled HTML table
        table_html = """
        <table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
            <thead style="background-color: #f2f2f2;">
                <tr>
        """
        
        for cell in header:
            table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{cell}</th>'
        
        table_html += "</tr></thead><tbody>"
        
        for i, row in enumerate(rows):
            bg_color = "#f9f9f9" if i % 2 == 0 else "#ffffff"
            table_html += f'<tr style="background-color: {bg_color};">'
            for cell in row:
                table_html += f'<td style="border: 1px solid #ddd; padding: 8px;">{cell}</td>'
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        enhanced_tables.append(table_html)
    
    return enhanced_tables

def display_code_blocks(parsed_response, collapsible=False):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        collapsible (bool): Whether to make code blocks collapsible
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        code_display = ""
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            code_display = f'<div id="code-block-{i}"></div>'
        else:
            # Fallback: display as preformatted text with basic styling
            code_display = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
        
        if collapsible:
            display(HTML(f"""
            <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; padding: 10px; background-color: #f0f0f0; 
                               font-weight: bold;">📝 Code Block {i} ({block['language']}) - Click to expand</summary>
                <div style="padding: 10px;">
                    {code_display}
                </div>
            </details>
            """))
        else:
            display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
            display(HTML(code_display))
        
        # Display actual code if using Code class
        if Code:
            display(Code(block['code'], language=block['language']))

def display_enhanced_tables(parsed_response):
    """
    Extract and display enhanced tables
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    tables = extract_tables(content)
    
    if not tables:
        print("No tables found in the response.")
        return
    
    for i, table_html in enumerate(tables, 1):
        display(HTML(f"<h4>📊 Table {i}</h4>"))
        display(HTML(table_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

def display_grok_analysis(parsed_response, extract_code=True, extract_tables=True, 
                         show_stats=True, collapsible=True):
    """
    Complete analysis and display of Grok response with all enhancements
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        extract_tables (bool): Whether to extract and display tables separately
        show_stats (bool): Whether to show token usage statistics
        collapsible (bool): Whether to make sections collapsible
    """
    
    # Show main response with LaTeX support
    display_grok_response(parsed_response, show_stats=show_stats, collapsible_stats=collapsible)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response, collapsible=collapsible)
    
    # Extract and display tables
    if extract_tables:
        print("\n" + "="*50)
        display(HTML("<h3>📊 Table Enhancement</h3>"))
        display_enhanced_tables(parsed_response)

def create_comprehensive_display(parsed_response, title="Grok Response Analysis"):
    """
    Create a comprehensive, collapsible display of all response components
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        title (str): Title for the display
    """
    
    # Main container with collapsible sections
    display(HTML(f"""
    <div style="border: 2px solid #007acc; border-radius: 10px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);">
        <h2 style="text-align: center; color: #007acc; margin: 0; padding: 20px;">
            🤖 {title}
        </h2>
    </div>
    """))
    
    # Stats section (collapsible)
    display_grok_response(parsed_response, show_stats=True, collapsible_stats=True)
    
    # Main content with LaTeX support
    content = parsed_response.get('message_content', '')
    display(HTML("""
    <details open style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
        <summary style="cursor: pointer; padding: 10px; background-color: #e8f4f8; 
                       font-weight: bold; font-size: 16px;">📝 Full Response Content</summary>
        <div style="padding: 10px;">
    """))
    
    display_content_with_latex(content)
    display(HTML("</div></details>"))
    
    # Code blocks (collapsible)
    code_blocks = extract_code_blocks(content)
    if code_blocks:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f0f8e8; 
                           font-weight: bold; font-size: 16px;">🔧 Code Blocks</summary>
            <div style="padding: 10px;">
        """))
        display_code_blocks(parsed_response, collapsible=False)
        display(HTML("</div></details>"))
    
    # Tables (collapsible)
    tables = extract_tables(content)
    if tables:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f8f0e8; 
                           font-weight: bold; font-size: 16px;">📊 Enhanced Tables</summary>
            <div style="padding: 10px;">
        """))
        display_enhanced_tables(parsed_response)
        display(HTML("</div></details>"))

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the enhanced helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with comprehensive analysis
    create_comprehensive_display(fibonacci_response, "Fibonacci Function Analysis")

# Enhanced usage examples
"""
# Basic usage with LaTeX support:
display_grok_response(your_parsed_response)

# Full analysis with collapsible sections:
display_grok_analysis(your_parsed_response, collapsible=True)

# Comprehensive display with all features:
create_comprehensive_display(your_parsed_response, "My Analysis")

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)

# Save code to file:
extract_and_save_code(your_parsed_response, "my_code.py")
""",  # Display math
        r'\$.*?\

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def extract_tables(content):
    """
    Extract and enhance markdown tables
    
    Args:
        content (str): Content with potential tables
        
    Returns:
        list: List of table HTML strings
    """
    # Pattern to match markdown tables
    table_pattern = r'(\|.*?\|(?:\r?\n|\r)\|.*?\|(?:(?:\r?\n|\r)\|.*?\|)*)'
    matches = re.findall(table_pattern, content, re.MULTILINE)
    
    enhanced_tables = []
    for match in matches:
        # Convert markdown table to HTML with better styling
        lines = match.strip().split('\n')
        if len(lines) < 2:
            continue
            
        # Parse header
        header = [cell.strip() for cell in lines[0].split('|')[1:-1]]
        
        # Skip separator line
        if len(lines) < 3:
            continue
            
        # Parse data rows
        rows = []
        for line in lines[2:]:
            if line.strip():
                row = [cell.strip() for cell in line.split('|')[1:-1]]
                rows.append(row)
        
        # Generate styled HTML table
        table_html = """
        <table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
            <thead style="background-color: #f2f2f2;">
                <tr>
        """
        
        for cell in header:
            table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{cell}</th>'
        
        table_html += "</tr></thead><tbody>"
        
        for i, row in enumerate(rows):
            bg_color = "#f9f9f9" if i % 2 == 0 else "#ffffff"
            table_html += f'<tr style="background-color: {bg_color};">'
            for cell in row:
                table_html += f'<td style="border: 1px solid #ddd; padding: 8px;">{cell}</td>'
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        enhanced_tables.append(table_html)
    
    return enhanced_tables

def display_code_blocks(parsed_response, collapsible=False):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        collapsible (bool): Whether to make code blocks collapsible
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        code_display = ""
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            code_display = f'<div id="code-block-{i}"></div>'
        else:
            # Fallback: display as preformatted text with basic styling
            code_display = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
        
        if collapsible:
            display(HTML(f"""
            <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; padding: 10px; background-color: #f0f0f0; 
                               font-weight: bold;">📝 Code Block {i} ({block['language']}) - Click to expand</summary>
                <div style="padding: 10px;">
                    {code_display}
                </div>
            </details>
            """))
        else:
            display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
            display(HTML(code_display))
        
        # Display actual code if using Code class
        if Code:
            display(Code(block['code'], language=block['language']))

def display_enhanced_tables(parsed_response):
    """
    Extract and display enhanced tables
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    tables = extract_tables(content)
    
    if not tables:
        print("No tables found in the response.")
        return
    
    for i, table_html in enumerate(tables, 1):
        display(HTML(f"<h4>📊 Table {i}</h4>"))
        display(HTML(table_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

def display_grok_analysis(parsed_response, extract_code=True, extract_tables=True, 
                         show_stats=True, collapsible=True):
    """
    Complete analysis and display of Grok response with all enhancements
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        extract_tables (bool): Whether to extract and display tables separately
        show_stats (bool): Whether to show token usage statistics
        collapsible (bool): Whether to make sections collapsible
    """
    
    # Show main response with LaTeX support
    display_grok_response(parsed_response, show_stats=show_stats, collapsible_stats=collapsible)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response, collapsible=collapsible)
    
    # Extract and display tables
    if extract_tables:
        print("\n" + "="*50)
        display(HTML("<h3>📊 Table Enhancement</h3>"))
        display_enhanced_tables(parsed_response)

def create_comprehensive_display(parsed_response, title="Grok Response Analysis"):
    """
    Create a comprehensive, collapsible display of all response components
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        title (str): Title for the display
    """
    
    # Main container with collapsible sections
    display(HTML(f"""
    <div style="border: 2px solid #007acc; border-radius: 10px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);">
        <h2 style="text-align: center; color: #007acc; margin: 0; padding: 20px;">
            🤖 {title}
        </h2>
    </div>
    """))
    
    # Stats section (collapsible)
    display_grok_response(parsed_response, show_stats=True, collapsible_stats=True)
    
    # Main content with LaTeX support
    content = parsed_response.get('message_content', '')
    display(HTML("""
    <details open style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
        <summary style="cursor: pointer; padding: 10px; background-color: #e8f4f8; 
                       font-weight: bold; font-size: 16px;">📝 Full Response Content</summary>
        <div style="padding: 10px;">
    """))
    
    display_content_with_latex(content)
    display(HTML("</div></details>"))
    
    # Code blocks (collapsible)
    code_blocks = extract_code_blocks(content)
    if code_blocks:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f0f8e8; 
                           font-weight: bold; font-size: 16px;">🔧 Code Blocks</summary>
            <div style="padding: 10px;">
        """))
        display_code_blocks(parsed_response, collapsible=False)
        display(HTML("</div></details>"))
    
    # Tables (collapsible)
    tables = extract_tables(content)
    if tables:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f8f0e8; 
                           font-weight: bold; font-size: 16px;">📊 Enhanced Tables</summary>
            <div style="padding: 10px;">
        """))
        display_enhanced_tables(parsed_response)
        display(HTML("</div></details>"))

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the enhanced helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with comprehensive analysis
    create_comprehensive_display(fibonacci_response, "Fibonacci Function Analysis")

# Enhanced usage examples
"""
# Basic usage with LaTeX support:
display_grok_response(your_parsed_response)

# Full analysis with collapsible sections:
display_grok_analysis(your_parsed_response, collapsible=True)

# Comprehensive display with all features:
create_comprehensive_display(your_parsed_response, "My Analysis")

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)

# Save code to file:
extract_and_save_code(your_parsed_response, "my_code.py")
""",      # Inline math
        r'\\begin\{.*?\}.*?\\end\{.*?\}',  # LaTeX environments
        r'\\[a-zA-Z]+\{.*?\}',  # LaTeX commands
        r'\\\(.*?\\\)',  # Old-style inline math
        r'\\\[.*?\\\]',  # Old-style display math
    ]
    
    has_latex = any(re.search(pattern, content, re.DOTALL) for pattern in latex_patterns)
    
    if has_latex:
        # Enable MathJax for LaTeX rendering with comprehensive delimiter support
        mathjax_config = """
        <script>
        if (typeof window.MathJax === 'undefined') {
            window.MathJax = {
                tex: {
                    inlineMath: [['

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def extract_tables(content):
    """
    Extract and enhance markdown tables
    
    Args:
        content (str): Content with potential tables
        
    Returns:
        list: List of table HTML strings
    """
    # Pattern to match markdown tables
    table_pattern = r'(\|.*?\|(?:\r?\n|\r)\|.*?\|(?:(?:\r?\n|\r)\|.*?\|)*)'
    matches = re.findall(table_pattern, content, re.MULTILINE)
    
    enhanced_tables = []
    for match in matches:
        # Convert markdown table to HTML with better styling
        lines = match.strip().split('\n')
        if len(lines) < 2:
            continue
            
        # Parse header
        header = [cell.strip() for cell in lines[0].split('|')[1:-1]]
        
        # Skip separator line
        if len(lines) < 3:
            continue
            
        # Parse data rows
        rows = []
        for line in lines[2:]:
            if line.strip():
                row = [cell.strip() for cell in line.split('|')[1:-1]]
                rows.append(row)
        
        # Generate styled HTML table
        table_html = """
        <table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
            <thead style="background-color: #f2f2f2;">
                <tr>
        """
        
        for cell in header:
            table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{cell}</th>'
        
        table_html += "</tr></thead><tbody>"
        
        for i, row in enumerate(rows):
            bg_color = "#f9f9f9" if i % 2 == 0 else "#ffffff"
            table_html += f'<tr style="background-color: {bg_color};">'
            for cell in row:
                table_html += f'<td style="border: 1px solid #ddd; padding: 8px;">{cell}</td>'
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        enhanced_tables.append(table_html)
    
    return enhanced_tables

def display_code_blocks(parsed_response, collapsible=False):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        collapsible (bool): Whether to make code blocks collapsible
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        code_display = ""
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            code_display = f'<div id="code-block-{i}"></div>'
        else:
            # Fallback: display as preformatted text with basic styling
            code_display = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
        
        if collapsible:
            display(HTML(f"""
            <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; padding: 10px; background-color: #f0f0f0; 
                               font-weight: bold;">📝 Code Block {i} ({block['language']}) - Click to expand</summary>
                <div style="padding: 10px;">
                    {code_display}
                </div>
            </details>
            """))
        else:
            display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
            display(HTML(code_display))
        
        # Display actual code if using Code class
        if Code:
            display(Code(block['code'], language=block['language']))

def display_enhanced_tables(parsed_response):
    """
    Extract and display enhanced tables
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    tables = extract_tables(content)
    
    if not tables:
        print("No tables found in the response.")
        return
    
    for i, table_html in enumerate(tables, 1):
        display(HTML(f"<h4>📊 Table {i}</h4>"))
        display(HTML(table_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

def display_grok_analysis(parsed_response, extract_code=True, extract_tables=True, 
                         show_stats=True, collapsible=True):
    """
    Complete analysis and display of Grok response with all enhancements
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        extract_tables (bool): Whether to extract and display tables separately
        show_stats (bool): Whether to show token usage statistics
        collapsible (bool): Whether to make sections collapsible
    """
    
    # Show main response with LaTeX support
    display_grok_response(parsed_response, show_stats=show_stats, collapsible_stats=collapsible)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response, collapsible=collapsible)
    
    # Extract and display tables
    if extract_tables:
        print("\n" + "="*50)
        display(HTML("<h3>📊 Table Enhancement</h3>"))
        display_enhanced_tables(parsed_response)

def create_comprehensive_display(parsed_response, title="Grok Response Analysis"):
    """
    Create a comprehensive, collapsible display of all response components
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        title (str): Title for the display
    """
    
    # Main container with collapsible sections
    display(HTML(f"""
    <div style="border: 2px solid #007acc; border-radius: 10px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);">
        <h2 style="text-align: center; color: #007acc; margin: 0; padding: 20px;">
            🤖 {title}
        </h2>
    </div>
    """))
    
    # Stats section (collapsible)
    display_grok_response(parsed_response, show_stats=True, collapsible_stats=True)
    
    # Main content with LaTeX support
    content = parsed_response.get('message_content', '')
    display(HTML("""
    <details open style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
        <summary style="cursor: pointer; padding: 10px; background-color: #e8f4f8; 
                       font-weight: bold; font-size: 16px;">📝 Full Response Content</summary>
        <div style="padding: 10px;">
    """))
    
    display_content_with_latex(content)
    display(HTML("</div></details>"))
    
    # Code blocks (collapsible)
    code_blocks = extract_code_blocks(content)
    if code_blocks:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f0f8e8; 
                           font-weight: bold; font-size: 16px;">🔧 Code Blocks</summary>
            <div style="padding: 10px;">
        """))
        display_code_blocks(parsed_response, collapsible=False)
        display(HTML("</div></details>"))
    
    # Tables (collapsible)
    tables = extract_tables(content)
    if tables:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f8f0e8; 
                           font-weight: bold; font-size: 16px;">📊 Enhanced Tables</summary>
            <div style="padding: 10px;">
        """))
        display_enhanced_tables(parsed_response)
        display(HTML("</div></details>"))

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the enhanced helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with comprehensive analysis
    create_comprehensive_display(fibonacci_response, "Fibonacci Function Analysis")

# Enhanced usage examples
"""
# Basic usage with LaTeX support:
display_grok_response(your_parsed_response)

# Full analysis with collapsible sections:
display_grok_analysis(your_parsed_response, collapsible=True)

# Comprehensive display with all features:
create_comprehensive_display(your_parsed_response, "My Analysis")

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)

# Save code to file:
extract_and_save_code(your_parsed_response, "my_code.py")
""", '

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def extract_tables(content):
    """
    Extract and enhance markdown tables
    
    Args:
        content (str): Content with potential tables
        
    Returns:
        list: List of table HTML strings
    """
    # Pattern to match markdown tables
    table_pattern = r'(\|.*?\|(?:\r?\n|\r)\|.*?\|(?:(?:\r?\n|\r)\|.*?\|)*)'
    matches = re.findall(table_pattern, content, re.MULTILINE)
    
    enhanced_tables = []
    for match in matches:
        # Convert markdown table to HTML with better styling
        lines = match.strip().split('\n')
        if len(lines) < 2:
            continue
            
        # Parse header
        header = [cell.strip() for cell in lines[0].split('|')[1:-1]]
        
        # Skip separator line
        if len(lines) < 3:
            continue
            
        # Parse data rows
        rows = []
        for line in lines[2:]:
            if line.strip():
                row = [cell.strip() for cell in line.split('|')[1:-1]]
                rows.append(row)
        
        # Generate styled HTML table
        table_html = """
        <table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
            <thead style="background-color: #f2f2f2;">
                <tr>
        """
        
        for cell in header:
            table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{cell}</th>'
        
        table_html += "</tr></thead><tbody>"
        
        for i, row in enumerate(rows):
            bg_color = "#f9f9f9" if i % 2 == 0 else "#ffffff"
            table_html += f'<tr style="background-color: {bg_color};">'
            for cell in row:
                table_html += f'<td style="border: 1px solid #ddd; padding: 8px;">{cell}</td>'
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        enhanced_tables.append(table_html)
    
    return enhanced_tables

def display_code_blocks(parsed_response, collapsible=False):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        collapsible (bool): Whether to make code blocks collapsible
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        code_display = ""
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            code_display = f'<div id="code-block-{i}"></div>'
        else:
            # Fallback: display as preformatted text with basic styling
            code_display = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
        
        if collapsible:
            display(HTML(f"""
            <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; padding: 10px; background-color: #f0f0f0; 
                               font-weight: bold;">📝 Code Block {i} ({block['language']}) - Click to expand</summary>
                <div style="padding: 10px;">
                    {code_display}
                </div>
            </details>
            """))
        else:
            display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
            display(HTML(code_display))
        
        # Display actual code if using Code class
        if Code:
            display(Code(block['code'], language=block['language']))

def display_enhanced_tables(parsed_response):
    """
    Extract and display enhanced tables
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    tables = extract_tables(content)
    
    if not tables:
        print("No tables found in the response.")
        return
    
    for i, table_html in enumerate(tables, 1):
        display(HTML(f"<h4>📊 Table {i}</h4>"))
        display(HTML(table_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

def display_grok_analysis(parsed_response, extract_code=True, extract_tables=True, 
                         show_stats=True, collapsible=True):
    """
    Complete analysis and display of Grok response with all enhancements
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        extract_tables (bool): Whether to extract and display tables separately
        show_stats (bool): Whether to show token usage statistics
        collapsible (bool): Whether to make sections collapsible
    """
    
    # Show main response with LaTeX support
    display_grok_response(parsed_response, show_stats=show_stats, collapsible_stats=collapsible)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response, collapsible=collapsible)
    
    # Extract and display tables
    if extract_tables:
        print("\n" + "="*50)
        display(HTML("<h3>📊 Table Enhancement</h3>"))
        display_enhanced_tables(parsed_response)

def create_comprehensive_display(parsed_response, title="Grok Response Analysis"):
    """
    Create a comprehensive, collapsible display of all response components
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        title (str): Title for the display
    """
    
    # Main container with collapsible sections
    display(HTML(f"""
    <div style="border: 2px solid #007acc; border-radius: 10px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);">
        <h2 style="text-align: center; color: #007acc; margin: 0; padding: 20px;">
            🤖 {title}
        </h2>
    </div>
    """))
    
    # Stats section (collapsible)
    display_grok_response(parsed_response, show_stats=True, collapsible_stats=True)
    
    # Main content with LaTeX support
    content = parsed_response.get('message_content', '')
    display(HTML("""
    <details open style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
        <summary style="cursor: pointer; padding: 10px; background-color: #e8f4f8; 
                       font-weight: bold; font-size: 16px;">📝 Full Response Content</summary>
        <div style="padding: 10px;">
    """))
    
    display_content_with_latex(content)
    display(HTML("</div></details>"))
    
    # Code blocks (collapsible)
    code_blocks = extract_code_blocks(content)
    if code_blocks:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f0f8e8; 
                           font-weight: bold; font-size: 16px;">🔧 Code Blocks</summary>
            <div style="padding: 10px;">
        """))
        display_code_blocks(parsed_response, collapsible=False)
        display(HTML("</div></details>"))
    
    # Tables (collapsible)
    tables = extract_tables(content)
    if tables:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f8f0e8; 
                           font-weight: bold; font-size: 16px;">📊 Enhanced Tables</summary>
            <div style="padding: 10px;">
        """))
        display_enhanced_tables(parsed_response)
        display(HTML("</div></details>"))

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the enhanced helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with comprehensive analysis
    create_comprehensive_display(fibonacci_response, "Fibonacci Function Analysis")

# Enhanced usage examples
"""
# Basic usage with LaTeX support:
display_grok_response(your_parsed_response)

# Full analysis with collapsible sections:
display_grok_analysis(your_parsed_response, collapsible=True)

# Comprehensive display with all features:
create_comprehensive_display(your_parsed_response, "My Analysis")

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)

# Save code to file:
extract_and_save_code(your_parsed_response, "my_code.py")
"""], ['\\\\(', '\\\\)']],
                    displayMath: [['$', '$'], ['\\\\[', '\\\\]']],
                    processEscapes: true,
                    processEnvironments: true,
                    processRefs: true,
                    macros: {
                        hbar: "\\\\hbar",
                        psi: "\\\\psi",
                        Psi: "\\\\Psi",
                        partial: "\\\\partial",
                        nabla: "\\\\nabla",
                        hat: ["\\\\hat{#1}", 1],
                        vec: ["\\\\vec{#1}", 1]
                    }
                },
                options: {
                    ignoreHtmlClass: 'tex2jax_ignore',
                    processHtmlClass: 'tex2jax_process'
                },
                startup: {
                    ready() {
                        MathJax.startup.defaultReady();
                        MathJax.startup.promise.then(() => {
                            console.log('MathJax loaded successfully');
                        });
                    }
                }
            };
        }
        </script>
        <script type="text/javascript" id="MathJax-script" async
            src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
        </script>
        """
        display(HTML(mathjax_config))
        
        # Pre-process content to handle old LaTeX notation better
        processed_content = content
        
        # Convert old-style delimiters to new ones for better compatibility
        # Handle display math first (longer patterns)
        processed_content = re.sub(r'\\\[(.*?)\\\]', r'$\1$', processed_content, flags=re.DOTALL)
        # Handle inline math
        processed_content = re.sub(r'\\\((.*?)\\\)', r'$\1

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def extract_tables(content):
    """
    Extract and enhance markdown tables
    
    Args:
        content (str): Content with potential tables
        
    Returns:
        list: List of table HTML strings
    """
    # Pattern to match markdown tables
    table_pattern = r'(\|.*?\|(?:\r?\n|\r)\|.*?\|(?:(?:\r?\n|\r)\|.*?\|)*)'
    matches = re.findall(table_pattern, content, re.MULTILINE)
    
    enhanced_tables = []
    for match in matches:
        # Convert markdown table to HTML with better styling
        lines = match.strip().split('\n')
        if len(lines) < 2:
            continue
            
        # Parse header
        header = [cell.strip() for cell in lines[0].split('|')[1:-1]]
        
        # Skip separator line
        if len(lines) < 3:
            continue
            
        # Parse data rows
        rows = []
        for line in lines[2:]:
            if line.strip():
                row = [cell.strip() for cell in line.split('|')[1:-1]]
                rows.append(row)
        
        # Generate styled HTML table
        table_html = """
        <table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
            <thead style="background-color: #f2f2f2;">
                <tr>
        """
        
        for cell in header:
            table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{cell}</th>'
        
        table_html += "</tr></thead><tbody>"
        
        for i, row in enumerate(rows):
            bg_color = "#f9f9f9" if i % 2 == 0 else "#ffffff"
            table_html += f'<tr style="background-color: {bg_color};">'
            for cell in row:
                table_html += f'<td style="border: 1px solid #ddd; padding: 8px;">{cell}</td>'
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        enhanced_tables.append(table_html)
    
    return enhanced_tables

def display_code_blocks(parsed_response, collapsible=False):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        collapsible (bool): Whether to make code blocks collapsible
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        code_display = ""
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            code_display = f'<div id="code-block-{i}"></div>'
        else:
            # Fallback: display as preformatted text with basic styling
            code_display = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
        
        if collapsible:
            display(HTML(f"""
            <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; padding: 10px; background-color: #f0f0f0; 
                               font-weight: bold;">📝 Code Block {i} ({block['language']}) - Click to expand</summary>
                <div style="padding: 10px;">
                    {code_display}
                </div>
            </details>
            """))
        else:
            display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
            display(HTML(code_display))
        
        # Display actual code if using Code class
        if Code:
            display(Code(block['code'], language=block['language']))

def display_enhanced_tables(parsed_response):
    """
    Extract and display enhanced tables
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    tables = extract_tables(content)
    
    if not tables:
        print("No tables found in the response.")
        return
    
    for i, table_html in enumerate(tables, 1):
        display(HTML(f"<h4>📊 Table {i}</h4>"))
        display(HTML(table_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

def display_grok_analysis(parsed_response, extract_code=True, extract_tables=True, 
                         show_stats=True, collapsible=True):
    """
    Complete analysis and display of Grok response with all enhancements
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        extract_tables (bool): Whether to extract and display tables separately
        show_stats (bool): Whether to show token usage statistics
        collapsible (bool): Whether to make sections collapsible
    """
    
    # Show main response with LaTeX support
    display_grok_response(parsed_response, show_stats=show_stats, collapsible_stats=collapsible)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response, collapsible=collapsible)
    
    # Extract and display tables
    if extract_tables:
        print("\n" + "="*50)
        display(HTML("<h3>📊 Table Enhancement</h3>"))
        display_enhanced_tables(parsed_response)

def create_comprehensive_display(parsed_response, title="Grok Response Analysis"):
    """
    Create a comprehensive, collapsible display of all response components
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        title (str): Title for the display
    """
    
    # Main container with collapsible sections
    display(HTML(f"""
    <div style="border: 2px solid #007acc; border-radius: 10px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);">
        <h2 style="text-align: center; color: #007acc; margin: 0; padding: 20px;">
            🤖 {title}
        </h2>
    </div>
    """))
    
    # Stats section (collapsible)
    display_grok_response(parsed_response, show_stats=True, collapsible_stats=True)
    
    # Main content with LaTeX support
    content = parsed_response.get('message_content', '')
    display(HTML("""
    <details open style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
        <summary style="cursor: pointer; padding: 10px; background-color: #e8f4f8; 
                       font-weight: bold; font-size: 16px;">📝 Full Response Content</summary>
        <div style="padding: 10px;">
    """))
    
    display_content_with_latex(content)
    display(HTML("</div></details>"))
    
    # Code blocks (collapsible)
    code_blocks = extract_code_blocks(content)
    if code_blocks:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f0f8e8; 
                           font-weight: bold; font-size: 16px;">🔧 Code Blocks</summary>
            <div style="padding: 10px;">
        """))
        display_code_blocks(parsed_response, collapsible=False)
        display(HTML("</div></details>"))
    
    # Tables (collapsible)
    tables = extract_tables(content)
    if tables:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f8f0e8; 
                           font-weight: bold; font-size: 16px;">📊 Enhanced Tables</summary>
            <div style="padding: 10px;">
        """))
        display_enhanced_tables(parsed_response)
        display(HTML("</div></details>"))

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the enhanced helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with comprehensive analysis
    create_comprehensive_display(fibonacci_response, "Fibonacci Function Analysis")

# Enhanced usage examples
"""
# Basic usage with LaTeX support:
display_grok_response(your_parsed_response)

# Full analysis with collapsible sections:
display_grok_analysis(your_parsed_response, collapsible=True)

# Comprehensive display with all features:
create_comprehensive_display(your_parsed_response, "My Analysis")

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)

# Save code to file:
extract_and_save_code(your_parsed_response, "my_code.py")
""", processed_content, flags=re.DOTALL)
        
        # Display with MathJax processing and a slight delay to ensure loading
        display(HTML(f'''
        <div class="tex2jax_process" style="font-size: 16px; line-height: 1.6;">
            {processed_content}
        </div>
        <script>
        setTimeout(function() {{
            if (typeof MathJax !== 'undefined' && MathJax.typesetPromise) {{
                MathJax.typesetPromise();
            }}
        }}, 100);
        </script>
        '''))
    else:
        # Regular markdown display
        display(Markdown(content))

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def extract_tables(content):
    """
    Extract and enhance markdown tables
    
    Args:
        content (str): Content with potential tables
        
    Returns:
        list: List of table HTML strings
    """
    # Pattern to match markdown tables
    table_pattern = r'(\|.*?\|(?:\r?\n|\r)\|.*?\|(?:(?:\r?\n|\r)\|.*?\|)*)'
    matches = re.findall(table_pattern, content, re.MULTILINE)
    
    enhanced_tables = []
    for match in matches:
        # Convert markdown table to HTML with better styling
        lines = match.strip().split('\n')
        if len(lines) < 2:
            continue
            
        # Parse header
        header = [cell.strip() for cell in lines[0].split('|')[1:-1]]
        
        # Skip separator line
        if len(lines) < 3:
            continue
            
        # Parse data rows
        rows = []
        for line in lines[2:]:
            if line.strip():
                row = [cell.strip() for cell in line.split('|')[1:-1]]
                rows.append(row)
        
        # Generate styled HTML table
        table_html = """
        <table style="border-collapse: collapse; width: 100%; margin: 10px 0;">
            <thead style="background-color: #f2f2f2;">
                <tr>
        """
        
        for cell in header:
            table_html += f'<th style="border: 1px solid #ddd; padding: 8px; text-align: left;">{cell}</th>'
        
        table_html += "</tr></thead><tbody>"
        
        for i, row in enumerate(rows):
            bg_color = "#f9f9f9" if i % 2 == 0 else "#ffffff"
            table_html += f'<tr style="background-color: {bg_color};">'
            for cell in row:
                table_html += f'<td style="border: 1px solid #ddd; padding: 8px;">{cell}</td>'
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        enhanced_tables.append(table_html)
    
    return enhanced_tables

def display_code_blocks(parsed_response, collapsible=False):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        collapsible (bool): Whether to make code blocks collapsible
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        code_display = ""
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            code_display = f'<div id="code-block-{i}"></div>'
        else:
            # Fallback: display as preformatted text with basic styling
            code_display = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
        
        if collapsible:
            display(HTML(f"""
            <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; padding: 10px; background-color: #f0f0f0; 
                               font-weight: bold;">📝 Code Block {i} ({block['language']}) - Click to expand</summary>
                <div style="padding: 10px;">
                    {code_display}
                </div>
            </details>
            """))
        else:
            display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
            display(HTML(code_display))
        
        # Display actual code if using Code class
        if Code:
            display(Code(block['code'], language=block['language']))

def display_enhanced_tables(parsed_response):
    """
    Extract and display enhanced tables
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    tables = extract_tables(content)
    
    if not tables:
        print("No tables found in the response.")
        return
    
    for i, table_html in enumerate(tables, 1):
        display(HTML(f"<h4>📊 Table {i}</h4>"))
        display(HTML(table_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

def display_grok_analysis(parsed_response, extract_code=True, extract_tables=True, 
                         show_stats=True, collapsible=True):
    """
    Complete analysis and display of Grok response with all enhancements
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        extract_tables (bool): Whether to extract and display tables separately
        show_stats (bool): Whether to show token usage statistics
        collapsible (bool): Whether to make sections collapsible
    """
    
    # Show main response with LaTeX support
    display_grok_response(parsed_response, show_stats=show_stats, collapsible_stats=collapsible)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response, collapsible=collapsible)
    
    # Extract and display tables
    if extract_tables:
        print("\n" + "="*50)
        display(HTML("<h3>📊 Table Enhancement</h3>"))
        display_enhanced_tables(parsed_response)

def create_comprehensive_display(parsed_response, title="Grok Response Analysis"):
    """
    Create a comprehensive, collapsible display of all response components
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        title (str): Title for the display
    """
    
    # Main container with collapsible sections
    display(HTML(f"""
    <div style="border: 2px solid #007acc; border-radius: 10px; margin: 20px 0; 
                background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);">
        <h2 style="text-align: center; color: #007acc; margin: 0; padding: 20px;">
            🤖 {title}
        </h2>
    </div>
    """))
    
    # Stats section (collapsible)
    display_grok_response(parsed_response, show_stats=True, collapsible_stats=True)
    
    # Main content with LaTeX support
    content = parsed_response.get('message_content', '')
    display(HTML("""
    <details open style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
        <summary style="cursor: pointer; padding: 10px; background-color: #e8f4f8; 
                       font-weight: bold; font-size: 16px;">📝 Full Response Content</summary>
        <div style="padding: 10px;">
    """))
    
    display_content_with_latex(content)
    display(HTML("</div></details>"))
    
    # Code blocks (collapsible)
    code_blocks = extract_code_blocks(content)
    if code_blocks:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f0f8e8; 
                           font-weight: bold; font-size: 16px;">🔧 Code Blocks</summary>
            <div style="padding: 10px;">
        """))
        display_code_blocks(parsed_response, collapsible=False)
        display(HTML("</div></details>"))
    
    # Tables (collapsible)
    tables = extract_tables(content)
    if tables:
        display(HTML("""
        <details style="margin: 10px 0; border: 1px solid #ddd; border-radius: 5px;">
            <summary style="cursor: pointer; padding: 10px; background-color: #f8f0e8; 
                           font-weight: bold; font-size: 16px;">📊 Enhanced Tables</summary>
            <div style="padding: 10px;">
        """))
        display_enhanced_tables(parsed_response)
        display(HTML("</div></details>"))

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the enhanced helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with comprehensive analysis
    create_comprehensive_display(fibonacci_response, "Fibonacci Function Analysis")

# Enhanced usage examples
"""
# Basic usage with LaTeX support:
display_grok_response(your_parsed_response)

# Full analysis with collapsible sections:
display_grok_analysis(your_parsed_response, collapsible=True)

# Comprehensive display with all features:
create_comprehensive_display(your_parsed_response, "My Analysis")

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)

# Save code to file:
extract_and_save_code(your_parsed_response, "my_code.py")
"""

SyntaxError: EOL while scanning string literal (28883168.py, line 65)

In [11]:
prompt = "解释薛定谔方程物理意义。请使用LaTeX数学符号。"
resp = ask_grok(prompt, client=client)
display_grok_response(resp)

### 🤖 Grok Response:

In [12]:
create_comprehensive_display(resp, "Physics Problem Analysis")